# 교차 검증, 하이퍼파라미터 튜닝
- 가장 좋은 모델 선정

In [1]:
import sklearn
sklearn.__version__

'1.6.1'

In [2]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np   # 수치 계산을 위한 numpy
from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트 분류기
from sklearn.svm import SVC  # 서포트 벡터 머신
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.metrics import accuracy_score  # 정확도 평가 지표

# 데이터 가져오기
data = pd.read_csv("train.csv")
data.head()

# 모델링에 사용할 특성(feature)과 타겟(taarget) 변수 선택
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']  # 승객 등급, 성별, 나이, 동반자 수, 요금, 탑승 항구
target = 'Survived'  # 생존 여부 (0: 사망, 1: 생존)
data['Survived'].value_counts() # 비율이 5:5 (x) ==> 클래스 불균형, Class Imbalanced

Survived
0    549
1    342
Name: count, dtype: int64

In [3]:
# 결측치 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# 데이터 전처리
- 고려사항 : 전체 데이터에 해도 무방한가? 아니면 훈련데이터에만 적용해야 하는가?
  + 전체 데이터에 해도 무방한가? 아니면 훈련데이터에만 적용해야 하는가?
  + 범주 데이터는 두가지 선택지 : One-Hot Encoder, Label Encoder (0, 1, 2) / 탐색적 데이터 분석 & 사회적인 통념

In [4]:
# data['Sex'].unique()
data['Sex'] = data['Sex'].map({'male' : 0, 'female' : 1}) # 성별, 남성=0, 여성=1
data['Embarked'] = data['Embarked'].map({'C' : 0, 'Q' : 1, 'S' : 2}) # 성별, 남성=0, 여성=1
data['Sex'], data['Embarked']

(0      0
 1      1
 2      1
 3      1
 4      0
       ..
 886    0
 887    1
 888    1
 889    0
 890    0
 Name: Sex, Length: 891, dtype: int64,
 0      2.0
 1      0.0
 2      2.0
 3      2.0
 4      2.0
       ... 
 886    2.0
 887    2.0
 888    2.0
 889    0.0
 890    1.0
 Name: Embarked, Length: 891, dtype: float64)

In [5]:
data['Embarked'].value_counts()

Embarked
2.0    644
0.0    168
1.0     77
Name: count, dtype: int64

# 데이터셋 분리

In [6]:
# 현재 고려하지 않은 feature 3개 : ID, Name, Cabin 
# -> ID 같은 유일값으로는 규칙을 찾을 수 없음 그래서 제외
# -> 결측치의 경우에는 채워서 평균을 구하는데 Cabin의 경우 결측치가 많아서 제외
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']  # 승객 등급, 성별, 나이, 동반자 수, 요금, 탑승 항구
target = 'Survived'  # 생존 여부 (0: 사망, 1: 생존)

X = data[features] # 특성 데이터
y = data[target] # 타겟 데이터

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, stratify=y # 층화추출
) 

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((668, 7), (223, 7), (668,), (223,))

# 피처 엔지니어링

In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 668 entries, 486 to 821
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    668 non-null    int64  
 1   Sex       668 non-null    int64  
 2   Age       537 non-null    float64
 3   SibSp     668 non-null    int64  
 4   Parch     668 non-null    int64  
 5   Fare      668 non-null    float64
 6   Embarked  666 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 41.8 KB


In [8]:
# 훈련데이터의 정보를 이용해서 테스트 데이터에 적용
# 훈련데이터에서 결측치를 채울 대표값 계산
age_mean = round(X_train['Age'].mean(),0)      # 승객 등급의 최빈값
embarked_mode = X_train['Embarked'].mode()  # 탑승 항구의 최빈값

# 훈련셋에서 결측치를 채울 대표값 계산
age_mean = X_train['Age'].mean()  # 나이의 평균값
fare_mean = X_train['Fare'].mean()  # 요금의 평균값
pclass_mode = X_train['Pclass'].mode()[0]  # 승객 등급의 최빈값
embarked_mode = X_train['Embarked'].mode()[0]  # 탑승 항구의 최빈값

# 훈련데이터 적용
X_train['Age'] = X_train['Age'].fillna(age_mean)
X_train['Embarked'] = X_train['Embarked'].fillna(embarked_mode)

# 테스트데이터 적용
X_test['Age'] = X_test['Age'].fillna(age_mean)
X_test['Embarked'] = X_test['Embarked'].fillna(embarked_mode)

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 668 entries, 486 to 821
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    668 non-null    int64  
 1   Sex       668 non-null    int64  
 2   Age       668 non-null    float64
 3   SibSp     668 non-null    int64  
 4   Parch     668 non-null    int64  
 5   Fare      668 non-null    float64
 6   Embarked  668 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 41.8 KB


In [10]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 223 entries, 157 to 639
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    223 non-null    int64  
 1   Sex       223 non-null    int64  
 2   Age       223 non-null    float64
 3   SibSp     223 non-null    int64  
 4   Parch     223 non-null    int64  
 5   Fare      223 non-null    float64
 6   Embarked  223 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 13.9 KB


- 정규화, 표준화, One-Hot Encoding 고려해야하나, 여기서는 안함
  + 이유 : 연속형 데이터로 볼만한 것아 없음 / 이산형 데이터(Count 데이터)가 대부분

# 모델링
- 각 모델별 하이퍼파라미터 후보군 정의
- 교차 검증
- 가장 최고의 모델을 선정

In [11]:
# 사용할 모델 정의
models = {
    'RandomForest': RandomForestClassifier(random_state=42),   # 랜덤 포레스트 분류기
    'SVM': SVC(random_state=42, probability=True),             # 서포트 벡터 머신
    'LogisticRegression': LogisticRegression(random_state=42)  # 로지스틱 회귀
}

# 각 모델별 하이퍼파라미터 후보군 정의
param_grid = {
    'RandomForest': [
        {'n_estimators': 100, 'max_depth': None},  # 트리 100개, 깊이 제한 없음
        {'n_estimators': 200, 'max_depth': 5},      # 트리 200개, 최대 깊이 5
        {'n_estimators': 300, 'max_depth': 10}      # 트리 200개, 최대 깊이 5
    ],
    'SVM': [
        {'C': 1.0, 'kernel': 'rbf'},    # RBF 커널, C=1.0
        {'C': 0.5, 'kernel': 'linear'}  # 선형 커널, C=0.5
    ],
    'LogisticRegression': [
        {'C': 1.0, 'max_iter': 1000},  # 기본 설정
        {'C': 0.1, 'max_iter': 1000}   # 더 강한 정규화
    ]
}

# 교차 검증을 통한 최적 모델 선택
best_score = 0 # 최고 성능 점수
best_model_name = None # 최고 성능 모델 이름
best_model = None # 최고 성능 모델 객체 


# 각 모델과 하이퍼파라미터 조합에 대해 교차검증 수행
for model_name, model in models.items():
    print(f"\n--- Testing {model_name} ---")
    for params in param_grid[model_name]:
        model.set_params(**params)  # 하이퍼파라미터 설정
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')  # 5-fold 교차검증
        mean_cv = np.mean(cv_scores)  # 평균 교차검증 점수
        print(f"Params: {params}, CV Accuracy: {mean_cv:.4f}")
        
        # 최고 성능 모델 업데이트
        if mean_cv > best_score:
            best_score = mean_cv
            best_model_name = model_name
            best_model = model.set_params(**params)

# 최종 선택된 모델로 테스트셋 평가
best_model.fit(X_train, y_train)  # 최적 모델 학습
y_pred = best_model.predict(X_test)  # 테스트셋 예측
test_acc = accuracy_score(y_test, y_pred)  # 테스트셋 정확도 계산

# 최종 결과 출력
print(f"\nBest Model: {best_model_name}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Test Set Accuracy: {test_acc:.4f}")


--- Testing RandomForest ---
Params: {'n_estimators': 100, 'max_depth': None}, CV Accuracy: 0.8234
Params: {'n_estimators': 200, 'max_depth': 5}, CV Accuracy: 0.8264
Params: {'n_estimators': 300, 'max_depth': 10}, CV Accuracy: 0.8368

--- Testing SVM ---
Params: {'C': 1.0, 'kernel': 'rbf'}, CV Accuracy: 0.6931
Params: {'C': 0.5, 'kernel': 'linear'}, CV Accuracy: 0.7934

--- Testing LogisticRegression ---
Params: {'C': 1.0, 'max_iter': 1000}, CV Accuracy: 0.8024
Params: {'C': 0.1, 'max_iter': 1000}, CV Accuracy: 0.8099

Best Model: RandomForest
Best CV Score: 0.8368
Test Set Accuracy: 0.7713


# 전처리 정보 저장 & 모델 저장 내보내기

In [12]:
import joblib # 모델 저장을 위한 라이브러리
import json # 전처리 정보 저장을 위한 json

# 전처리 정보 저장
preprocessing_info = {
    'age_mean': float(age_mean),
    'fare_mean': float(fare_mean),
    'pclass_mode': int(pclass_mode),
    'embarked_mode': int(embarked_mode),
    'features': features,
    'sex_mapping': {'male': 0, 'female': 1},
    'embarked_mapping': {'C': 0, 'Q': 1, 'S': 2}
}

# 모델과 전처리 정보 저장
joblib.dump(best_model, 'titanic_model.joblib')

# 훈련데이터의 정보를 추가했음
with open('preprocessing_info.json', 'w') as f:
    json.dump(preprocessing_info, f)

# 최종 결과 출력
print(f"\nBest Model: {best_model_name}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Test Set Accuracy: {test_acc:.4f}")
print("\nModel and preprocessing information have been saved.")


Best Model: RandomForest
Best CV Score: 0.8368
Test Set Accuracy: 0.7713

Model and preprocessing information have been saved.


전체 모델 만들기 복습 및 Predict Calorie Expenditure 캐글 대회 데이터에도 적용
- 코드 추가할 것 : 교차 검증 시, 시간 측정 하기
- 하이퍼파라미터 튜닝 갯수를 늘릴 때 마다 시간이 제법 많이 소요됨 확인